In [ ]:
!pip install tensorflow
!pip install keras_unet
!pip install opencv-contrib-python
!pip install matplotlib

In [ ]:
import tensorflow as tf
from matplotlib import pyplot as plt

tf.__version__

In [ ]:
from keras_unet.metrics import iou, iou_thresholded

model_path = f"models/model.1655852129"
model = tf.keras.models.load_model(model_path,
                                   custom_objects=dict(iou_thresholded=iou_thresholded, iou=iou))

In [ ]:
model.summary()

In [ ]:
import cv2 as cv
path = 'images/training/2image.png'

image = cv.imread(path)
image = image[...,::-1] / 255

plt.imshow(image)
print(image.shape)

In [ ]:
from imsplit import imsplit
crops = imsplit(image, 256, step=64)

len(crops)

In [ ]:
batch = np.array([crop[0] for crop in crops])

batch.shape, batch.dtype, batch.max()

In [ ]:
result_batch = model.predict(batch)

In [ ]:
from imsplit import replace_segments, imjoin_average, imjoin_min, imjoin_max

result_crops = replace_segments(crops=crops, new_segments=result_batch)
result_image = imjoin_average(result_crops)

plt.imshow(np.squeeze(result_image), cmap='gray')
plt.show()

In [ ]:
from PIL import Image

path = 'image.png'
im = Image.fromarray(np.squeeze((result_image * 255).astype(np.uint8))).save(path)